In [1]:
%%shell
if ! command -v julia 3>&1 > /dev/null
then
    wget 'https://julialang-s3.julialang.org/bin/linux/x64/1.3/julia-1.3.1-linux-x86_64.tar.gz' -O julia.tar.gz
    tar -x -f julia.tar.gz -C /usr/local --strip-components 1
    rm julia.tar.gz
fi
julia -e 'using Pkg; pkg"add IJulia; add BenchmarkTools; add CuArrays; precompile;"'
echo 'Done'

  Updating registry at `~/.julia/registries/General`
  Updating git-repo `https://github.com/JuliaRegistries/General.git`
 Resolving package versions...
  Updating `~/.julia/environments/v1.3/Project.toml`
 [no changes]
  Updating `~/.julia/environments/v1.3/Manifest.toml`
 [no changes]
 Resolving package versions...
  Updating `~/.julia/environments/v1.3/Project.toml`
 [no changes]
  Updating `~/.julia/environments/v1.3/Manifest.toml`
 [no changes]
 Resolving package versions...
  Updating `~/.julia/environments/v1.3/Project.toml`
 [no changes]
  Updating `~/.julia/environments/v1.3/Manifest.toml`
 [no changes]
Precompiling project...
Done


# Julia 檔案處理與資料庫連線

## Day 014 作業1：讀取 Nested Dict 內的資料

Day 010 時我們介紹了字典 (Dict)，字典內的資料可以是巢狀 (nested) 的，包含較複雜的資料階層結構。在今天的下載檔 CityCountyData.json 是台灣各縣市鄉鎮區及路名的中英文資料，檔案為 JSON 格式，範例內容如下：

```json
    {
        "CityName": "臺北市",
        "CityEngName": "Taipei City",
        "AreaList": [
            {
                "ZipCode": "100",
                "AreaName": "中正區",
                "AreaEngName": "Zhongzheng Dist."
            },
            ...
    }
```

作業內容為讀取 JSON 檔案，並列出台北市所有行政區的中英文名稱。範例答案將以 JSON.jl 套件作為示範。

檔案資料來源：[台灣 縣市，鄉鎮，地址 中英文 JSON](https://github.com/donma/TaiwanAddressCityAreaRoadChineseEnglishJSON)

列出所有台北市行政區的中英文名稱

In [2]:
!julia -e'\
using HTTP,JSON                                                                                                    ;\
url = "https://github.com/donma/TaiwanAddressCityAreaRoadChineseEnglishJSON/raw/master/AllData.json"               ;\
response = HTTP.get(url)                                                                                           ;\
html = JSON.parse(String(response.body))                                                                           ;\
dict = Dict(html)                                                                                                  ;\
for i in html                                                                                                      ;\
     if i["CityName"] ==  "臺北市"                                                                                  ;\
          for j in i["AreaList"]                                                                                   ;\
              println(j["AreaName"],"   ",j["AreaEngName"])                                                        ;\
              end                                                                                                  ;\
      end                                                                                                          ;\
end                                                                                                                ;\'

中正區   Zhongzheng Dist.
大同區   Datong Dist.
中山區   Zhongshan Dist.
松山區   Songshan Dist.
大安區   Da’an Dist.
萬華區   Wanhua Dist.
信義區   Xinyi Dist.
士林區   Shilin Dist.
北投區   Beitou Dist.
內湖區   Neihu Dist.
南港區   Nangang Dist.
文山區   Wenshan Dist.


## 作業2：將字典資料存為 JSON 檔

作業2請產生字典 (Dict) 資料，並將字典資料存為 JSON 格式。請自行產生字典，也可使用下列的字串資料計算字數 (Day 010 作業程式)。

【提示】可以參考今天範例程式中將陣列資料存為 JSON 格式的部分。

In [3]:
!julia -e '\
str = "永和有永和路，中和也有永和路，                                 ;\
中和有中和路，永和也有中和路；                                        ;\
中和的中和路有接永和的中和路，                                        ;\
永和的永和路沒接中和的永和路；                                        ;\
永和的中和路有接永和的永和路，                                        ;\
中和的永和路沒接中和的中和路。                                        ;\
永和有中正路，中和也有中正路；                                        ;\
永和的中正路用景平路接中和的中正路。                                   ;\
永和有中山路，中和也有中山路；                                        ;\
永和的中山路直接接上了中和的中山路。                                   ;\
永和的中正路接上了永和的中山路；                                      ;\
中和的中正路卻不接中和的中山路。                                      ;\
中正橋下來不是中正路，但永和有中正路；                                 ;\
秀朗橋下來不是秀朗路，但永和有秀朗路。                                 ;\
永福橋下來不是永福路，永和沒有永福路；                                 ;\
福和橋下來不是福和路，但福和路接的卻是永福橋。                          ;\
中和中和路永和永和路                                                  ;\
永和中和路中和永和路                                                  ;\
中和中山路永和中正路                                                  ;\
永和中山路中和中正路"                                                 ;\
                                                                    ;\
d = Dict{String, Int64}()                                           ;\
for c ∈ str                                                         ;\
    d[string(c)] = get(d, string(c), 0) + 1                         ;\
end                                                                 ;\
print(d)                                                            ;\'

Dict("接" => 10,"上" => 2," " => 734,"是" => 5,"也" => 4,"直" => 1,"秀" => 3,"和" => 54,"來" => 4,"，" => 11,"沒" => 3,"永" => 31,"的" => 17,"平" => 1,"但" => 3,"正" => 11,"不" => 5,"用" => 1,"中" => 43,"橋" => 5,"福" => 7,"路" => 41,"山" => 8,"；" => 7,"。" => 6,"卻" => 2,"下" => 4,"了" => 2,"有" => 13,";" => 19,"景" => 1,"朗" => 3)